In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp
import os
from dotenv import load_dotenv

import pymysql
# 상위 폴더(../)에 있는 `.env` 파일 로드
dotenv_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".env"))
load_dotenv(dotenv_path)
## mysql 연결
os.getenv('user_id')
os.getenv('user_password')

mysql_url = f"jdbc:mysql://{os.getenv('host_ip')}:3306/{os.getenv('DATABASE')}?useSSL=false&allowPublicKeyRetrieval=true&useUnicode=true&characterEncoding=UTF-8"

# SparkSession 생성
spark = SparkSession.builder \
    .appName("HDFS to MySQL") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()

# HDFS에 있는 모든 Parquet 파일 로드
df = spark.read.parquet("hdfs:///data/")
print(df.count())

## 시간 형식 수정(yyyy.mm.dd HH:MM:SS)
convert_cols = ['comment_publish_date', 'publish_date']
for c in convert_cols:
    df = df.withColumn(c
                     , to_timestamp(col('comment_publish_date'), "yyyy-MM-dd'T'HH:mm:ss'Z'")
                     )
    
## 결측치 제거
df = df.dropna()

# PyMySQL로 MySQL 연결 (SQL 실행을 위해)
conn = pymysql.connect(
    host=os.getenv('host_ip'),
    user=os.getenv('user_id'),
    password=os.getenv('user_password'),
    database=os.getenv('DATABASE'),
)

cursor = conn.cursor()

# ✅ MySQL 테이블 `TRUNCATE`
truncate_query = "TRUNCATE TABLE tbCrawled_Youtube"
cursor.execute(truncate_query)

conn.commit()
cursor.close()
conn.close()

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/local/lib/mysql-connector-java-5.1.49-bin.jar pyspark-shell'
df.write.format('jdbc')\
    .options(
        url = mysql_url
        , driver = 'com.mysql.jdbc.Driver'
        , dbtable = 'tbCrawled_Youtube'
        , user = os.getenv('user_id')
        , password = os.getenv('user_password')
    )\
    .mode('append')\
    .save()

spark.stop()

136568


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:32927)
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:32927)
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceb

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:32927)
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/Project/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_20578/1347298694.py", line 17, in <module>
    spark = SparkSession.builde

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:32927)
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/Project/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_20578/1347298694.py", line 17, in <module>
    spark = SparkSession.builde

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:32927)

In [4]:
spark.stop()

In [1]:
import os
from pyspark.sql import SparkSession

host_ip = '15.168.221.131'
DATABASE = 'SNS_DB'
user_id = 'lab13'
user_password = 'lab13'

# 환경변수 설정
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/local/lib/mysql-connector-java-5.1.49-bin.jar pyspark-shell'

ss = SparkSession\
            .builder\
            .appName('MySQL')\
            .config('spark.executor.memory', '6g')\
            .config('spark.driver.memory', '6g')\
            .getOrCreate()

mysql_url = f'jdbc:mysql://{host_ip}:3306/{DATABASE}?useSSL=false&allowPublicKeyRetrieval=true'
ss.read.format('jdbc')\
            .options(
                url = mysql_url
                , driver = 'com.mysql.jdbc.Driver'
                , dbtable = 'tbCrawled_Youtube'
                , user = user_id
                , password = user_password
                    )\
        .load()\
        .show()

25/02/04 14:48:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+---+--------+-----+------------+------------+-------+----------+--------------------+-----------+
| id|video_id|title|publish_date|channel_name|comment|like_count|comment_publish_date|inserted_at|
+---+--------+-----+------------+------------+-------+----------+--------------------+-----------+
+---+--------+-----+------------+------------+-------+----------+--------------------+-----------+



In [2]:
ss.stop()